# Classification Algorithm:

- Data Description.

- Goal.
    
Using logistic regression to understand the demographics and other characteristics of a bank customers' that accept a credit card offer and that do not accept a credit card.

Data cleaning, wrangling and EDA on a database relative to an automobile portfolio from an insurance company. The goal is to analyze the data and define a model that fits to do predictions. The the variable 'total claim amount' should be considered as a target.

## Importing used packages

In [ ]:
import pymysql
from sqlalchemy import create_engine
from scipy import stats
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
import getpass
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/mid_project'
engine = create_engine(connection_string)

import warnings
warnings.filterwarnings('ignore')

## Getting Data

In [ ]:
query = '''SELECT *
           FROM creditcardmarketing
           ;'''

data = pd.read_sql_query(query, engine)
data

## Cleaning/Wrangling/EDA

- Change headers names.
- Deal with NaN values.
- Categorical Features.
- Numerical Features.
- Exploration.

In [ ]:
data.shape, data.dtypes

### Creating a function to clean the headers
- Renaming the columns so they follow the PE8 (snake case)
- Renaming the columns taking out non alphanumeric characters

In [ ]:
def clean_header(df):
        pattern = '[\W]'
        df.columns = [x.lower().replace(" ", "_") for x in df.columns]
        df.columns = [re.sub(pattern, "", x) for x in df.columns]
        return df

In [ ]:
# Another way to clean the headers:
def clean_headers(df):
    df.columns = [x.lower().replace(" ", "_") for x in df.columns]
    num = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    char = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm'
            , 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    keep = ['_']
    cols = df.columns
    new_col_names = []
    for col in cols:
        new_col = ''
        for alphabet in col:
            if (alphabet in num) or (alphabet in char) or (alphabet in keep):
                new_col += alphabet
        new_col_names.append(new_col)

    df.columns = new_col_names
    return df

In [ ]:
data = clean_headers(data) #or clean_header(data)
data.columns

# If using the function clean_header(data)
# it would be also necessary rename the remaining column with the character 'ï':
# data.rename(columns={'ïcustomer_number':'customer_number'}, inplace=True)

### NaN values

In [ ]:
# Checking NaN values
data.isna().sum()

### Numerical Features

In [ ]:
# summary of the data
data.describe().T.apply(lambda x: round(x,2))

In [ ]:
print(data['customer_number'].nunique())

In [ ]:
# As the df has 17976 rows and 'nunique() = 17976' for column 'customer_number' this column was setted as an index
data.set_index('customer_number')

### Categorical Features

In [ ]:
for col in data.select_dtypes('object'):
    print(data[col].value_counts(), '\n')

## Exploration

In [ ]:
# checking how many 'offer_accepted' by 'income_level'
pd.crosstab(data['offer_accepted'], data['income_level'])

In [ ]:
# checking how many 'offer_accepted' by 'credit_rating'
pd.crosstab(data['offer_accepted'], data['credit_rating'])

In [ ]:
# checking how many 'offer_accepted' by 'own_your_home'
pd.crosstab(data['offer_accepted'], data['own_your_home'])

In [ ]:
# check the correlations
corr_matrix=data.corr(method='pearson')  # default
fig, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
# As the columns 'average_balance' and 'q1/2/3/4_balance' showed a high correlation they will be dropped
data = data.drop(columns=['q1_balance', 'q2_balance', 'q3_balance', 'q4_balance'])

In [ ]:
for col in data.select_dtypes(np.number):
    if col == 'customer_number':
        pass
    else:
        sns.distplot(data[col])
        plt.show()

In [ ]:
for col in data.select_dtypes(np.number):
    if col == 'customer_number':
        pass
    else:
        sns.boxplot(data[col])
        plt.show()

## Processing Data

- Dealing with outliers.
- Normalization.
- Encoding Categorical Data.
- Splitting into train set and test set.

### Feature Engineering

In [ ]:
def boxcox_transform(df):
    numeric_cols = df.select_dtypes(np.number).columns
    _ci = {column: None for column in numeric_cols}
    for column in numeric_cols:
        # since i know any columns should take negative numbers, to avoid -inf in df
        df[column] = np.where(df[column]<=0, np.NAN, df[column]) 
        df[column] = df[column].fillna(df[column].mean())
        transformed_data, ci = stats.boxcox(df[column])
        df[column] = transformed_data
        _ci[column] = [ci] 
    return df, _ci

In [ ]:
# I would like to apply the transormation only on the column 'average_balance'
# , bc the other numerical ones have discrete occurrences
# But is it boxcox a the proper transformation to apply once the curve has 2 humps?

data['average_balance'] = np.where(data['average_balance']<=0, np.NAN, data['average_balance']) 
data['average_balance'] = data['average_balance'].fillna(data['average_balance'].mean())
transformed_data, ci = stats.boxcox(data['average_balance'])
data['average_balance'] = transformed_data
#_ci['average_balance'] = [ci]

sns.distplot(data['average_balance'])
plt.show()

In [ ]:
upper = np.percentile(data['average_balance'],75)
lower = np.percentile(data['average_balance'],25)
iqr = upper - lower
upper_limit = upper + (1.5 * iqr)
lower_limit = lower - (1.5 * iqr)
data = data[(data['average_balance']>lower_limit) & (data['average_balance']<upper_limit)]

sns.boxplot(data['average_balance'])
plt.show()

In [ ]:
# checking the size of the data loss after removing outliers
data.shape

### Train the model

In [ ]:
y = data['offer_accepted']
X = data.drop('offer_accepted', axis=1)

In [ ]:
X = pd.get_dummies(X)
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
classification = LogisticRegression(random_state=42, max_iter=1000)

classification.fit(X_train, y_train)

### Evaluate the model

In [ ]:
classification.score(X_test, y_test)

In [ ]:
# to get predictions
predictions = classification.predict(X_test)
predictions

In [ ]:
confusion_matrix(y_test, predictions) # normalize : {'true', 'pred', 'all'}

In [ ]:
cf_matrix = confusion_matrix(y_test, predictions, normalize='all')
cf_matrix

In [ ]:
sns.heatmap(cf_matrix, annot=True, fmt='.2%', cmap='Blues')

In [ ]:
group_names = ['True A', 'False A', 'False B', 'True B']
group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')

In [ ]:
# after get dummies
data.corr()['offer_accepted'].sort_values(ascending=False)